In [1]:
import urllib.request as urllib
import json

url = 'https://www.datos.gov.co/resource/gt2j-8ykr.json'

content=urllib.urlopen(url) 
datastore = json.load(content)

datos = []
for linea in datastore: 
    claves = list(linea.keys())
    valores = []
    
    for clave in claves:
        valores.append(linea[clave])
        
    datos.append(linea)
print(linea)



{'id_de_caso': '1000', 'fecha_de_notificaci_n': '2020-03-27T00:00:00.000', 'c_digo_divipola': '11001', 'ciudad_de_ubicaci_n': 'Bogotá D.C.', 'departamento': 'Bogotá D.C.', 'atenci_n': 'Recuperado', 'edad': '30', 'sexo': 'F', 'tipo': 'En estudio', 'estado': 'Leve', 'fis': '2020-03-18T00:00:00.000', 'fecha_diagnostico': '2020-04-01T00:00:00.000', 'fecha_recuperado': '2020-04-03T00:00:00.000', 'fecha_reporte_web': '2020-04-01T00:00:00.000', 'tipo_recuperaci_n': 'PCR', 'codigo_departamento': '11', 'pertenencia_etnica': 'Otro'}


In [2]:
# forma correcta de recorrer json con lineas que posiblemente contienen informacion faltante
# jemm, datastore contiene la lectura

def nueva_llave(linea):
    
    global diccionario
    
    claves_ln = list(linea.keys())
    claves_dc = list(diccionario.keys())
    
    if set(claves_ln) != set(claves_dc):
        claves_nv = list(set(claves_ln) - set(claves_dc))
        
        for cl in claves_nv:
            diccionario[cl] = []
    
def nueva_valor(linea):
    
    global diccionario
    
    for cl in diccionario:
        if cl in list(linea.keys()):
            diccionario[cl].append(linea[cl])
        else:
            diccionario[cl].append(None)
    

i = 0
diccionario = {}
for linea in datastore: 
    nueva_llave(linea)
    nueva_valor(linea)
        
    #i += 1
    #if i > 30:
    #    break



In [3]:
esquema = [cl for cl in diccionario]

print(esquema)
    

['sexo', 'codigo_pais', 'fecha_diagnostico', 'edad', 'fis', 'ciudad_de_ubicaci_n', 'fecha_reporte_web', 'codigo_departamento', 'departamento', 'atenci_n', 'fecha_recuperado', 'c_digo_divipola', 'estado', 'id_de_caso', 'pa_s_de_procedencia', 'pertenencia_etnica', 'fecha_de_notificaci_n', 'tipo_recuperaci_n', 'tipo', 'nombre_grupo_etnico', 'fecha_de_muerte']


In [4]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

appName = "Python PySpark Parsing Dictionary as DataFrame"

# Create Spark session
spark = SparkSession.builder \
    .appName(appName) \
    .master('local') \
    .getOrCreate()



In [5]:
# Create data frame
df = spark.createDataFrame(datos)


C:\Users\User\arbol\spark-hadoop\python\pyspark\sql\session.py:375: UserWarning: inferring schema from dict is deprecated,please use pyspark.sql.Row instead
  warnings.warn("inferring schema from dict is deprecated,"


In [6]:
df.select(['fecha_recuperado', 'fecha_diagnostico', 'fis', 'id_de_caso', 'ciudad_de_ubicaci_n', 'pa_s_de_procedencia',]).show(30)

+--------------------+--------------------+--------------------+----------+-------------------+-------------------+
|    fecha_recuperado|   fecha_diagnostico|                 fis|id_de_caso|ciudad_de_ubicaci_n|pa_s_de_procedencia|
+--------------------+--------------------+--------------------+----------+-------------------+-------------------+
|2020-03-13T00:00:...|2020-03-06T00:00:...|2020-02-27T00:00:...|         1|        Bogotá D.C.|             ITALIA|
|2020-03-19T00:00:...|2020-03-09T00:00:...|2020-03-04T00:00:...|         2|Guadalajara de Buga|             ESPAÑA|
|2020-03-15T00:00:...|2020-03-09T00:00:...|2020-02-29T00:00:...|         3|           Medellín|             ESPAÑA|
|2020-03-26T00:00:...|2020-03-11T00:00:...|2020-03-06T00:00:...|         4|           Medellín|               null|
|2020-03-23T00:00:...|2020-03-11T00:00:...|2020-03-08T00:00:...|         5|           Medellín|               null|
|2020-03-26T00:00:...|2020-03-11T00:00:...|2020-03-06T00:00:...|        

In [7]:
import pandas as pd
df = pd.DataFrame(datos)
df.head()

,id_de_caso,fecha_de_notificaci_n,c_digo_divipola,ciudad_de_ubicaci_n,departamento,atenci_n,edad,sexo,tipo,estado,...,fis,fecha_diagnostico,fecha_recuperado,fecha_reporte_web,tipo_recuperaci_n,codigo_departamento,codigo_pais,pertenencia_etnica,nombre_grupo_etnico,fecha_de_muerte
0,1,2020-03-02T00:00:00.000,11001,Bogotá D.C.,Bogotá D.C.,Recuperado,19,F,Importado,Leve,...,2020-02-27T00:00:00.000,2020-03-06T00:00:00.000,2020-03-13T00:00:00.000,2020-03-06T00:00:00.000,PCR,11,380,Otro,NaN,NaN
1,2,2020-03-06T00:00:00.000,76111,Guadalajara de Buga,Valle del Cauca,Recuperado,34,M,Importado,Leve,...,2020-03-04T00:00:00.000,2020-03-09T00:00:00.000,2020-03-19T00:00:00.000,2020-03-09T00:00:00.000,PCR,76,724,Otro,NaN,NaN
2,3,2020-03-07T00:00:00.000,05001,Medellín,Antioquia,Recuperado,50,F,Importado,Leve,...,2020-02-29T00:00:00.000,2020-03-09T00:00:00.000,2020-03-15T00:00:00.000,2020-03-09T00:00:00.000,PCR,5,724,Otro,NaN,NaN
3,4,2020-03-09T00:00:00.000,05001,Medellín,Antioquia,Recuperado,55,M,Relacionado,Leve,...,2020-03-06T00:00:00.000,2020-03-11T00:00:00.000,2020-03-26T00:00:00.000,2020-03-11T00:00:00.000,PCR,5,NaN,Otro,NaN,NaN
4,5,2020-03-09T00:00:00.000,05001,Medellín,Antioquia,Recuperado,25,M,Relacionado,Leve,...,2020-03-08T00:00:00.000,2020-03-11T00:00:00.000,2020-03-23T00:00:00.000,2020-03-11T00:00:00.000,PCR,5,NaN,Otro,NaN,NaN


In [ ]:

for col in df.columns:
    print(col, df.select(df[col]).show(1), '\n')


In [ ]:
columnas = sorted(df.columns)

for col in df.columns:
    print(df.describe([col]).show())